In [1]:
import pandas as pd
import polars as pl
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import gc
import joblib

pd.set_option('display.max_columns', None)

dataPath = "/kaggle/input/home-credit-credit-risk-model-stability/"

In [2]:
# helper functions
# copied from
# https://www.kaggle.com/code/liamhealy/lightgbm-feature-importance-all-datasets

def set_table_dtypes(df: pl.DataFrame) -> pl.DataFrame:
    # implement here all desired dtypes for tables
    # the following is just an example
    for col in df.columns:
        # last letter of column name will help you determine the type
        if col[-1] in ("P", "A"):
            df = df.with_columns(pl.col(col).cast(pl.Float64).alias(col))
    return df

def convert_strings(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:  
        if df[col].dtype.name in ['object', 'string']:
            df[col] = df[col].astype("string").astype('category')
            current_categories = df[col].cat.categories
            new_categories = current_categories.to_list() + ["Unknown"]
            new_dtype = pd.CategoricalDtype(categories=new_categories, ordered=True)
            df[col] = df[col].astype(new_dtype)
    return df

def from_polars_to_pandas(case_ids: pl.DataFrame) -> pl.DataFrame:
    return (
        data.filter(pl.col("case_id").is_in(case_ids))[["case_id", "WEEK_NUM", "target"]].to_pandas(),
        data.filter(pl.col("case_id").is_in(case_ids))[cols_pred].to_pandas(),
        data.filter(pl.col("case_id").is_in(case_ids))["target"].to_pandas()
    )

def summary(df):
    summ = pd.DataFrame(df.dtypes, columns=['data type'])
    summ['#total'] = df.shape[0]
    summ['#missing'] = df.isnull().sum().values 
    summ['%missing'] = df.isnull().sum().values / len(df)* 100
    summ['#unique'] = df.nunique().values
    summ['#duplicates'] = summ['#total'] - summ['#unique']
    desc = pd.DataFrame(df.describe(include='all').transpose())
    summ['min'] = desc['min'].values
    summ['max'] = desc['max'].values
    return summ

def gini_stability(base, w_fallingrate=88.0, w_resstd=-0.5):
    gini_in_time = base.loc[:, ["WEEK_NUM", "target", "score"]]\
        .sort_values("WEEK_NUM")\
        .groupby("WEEK_NUM")[["target", "score"]]\
        .apply(lambda x: 2*roc_auc_score(x["target"], x["score"])-1).tolist()
    
    x = np.arange(len(gini_in_time))
    y = gini_in_time
    a, b = np.polyfit(x, y, 1)
    y_hat = a*x + b
    residuals = y - y_hat
    res_std = np.std(residuals)
    avg_gini = np.mean(gini_in_time)
    return avg_gini + w_fallingrate * min(0, a) + w_resstd * res_std

def drop_outliers(df, field_name):
    iqr = 1.5 * (np.percentile(df[field_name], 75) - np.percentile(df[field_name], 25))
    df.drop(df[df[field_name] > (iqr + np.percentile(df[field_name], 75))].index, inplace=True)
    df.drop(df[df[field_name] < (np.percentile(df[field_name], 25) - iqr)].index, inplace=True)

In [3]:
# optimized & saveds ingles models by optuna
# for more details
# https://www.kaggle.com/code/mzaoualim/testing-error

model_static_cb_0 = joblib.load('/kaggle/input/models-opt/model_opt_static_cb_0.pkl')
model_static_0_1 = joblib.load('/kaggle/input/models-opt/model_opt_static_0_1.pkl')
model_stage_2 = joblib.load('/kaggle/input/models-opt/model_stage_2_opt.pkl')

In [4]:
# training columns lists

static_cb_0_cols = [
     'assignmentdate_238D',
     'assignmentdate_4527235D',
     'assignmentdate_4955616D',
     'birthdate_574D',
     'contractssum_5085716L',
     'dateofbirth_337D',
     'dateofbirth_342D',
     'days120_123L',
     'days180_256L',
     'days30_165L',
     'days360_512L',
     'days90_310L',
     'description_5085714M',
     'education_1103M',
     'education_88M',
     'firstquarter_103L',
     'for3years_128L',
     'for3years_504L',
     'for3years_584L',
     'formonth_118L',
     'formonth_206L',
     'formonth_535L',
     'forquarter_1017L',
     'forquarter_462L',
     'forquarter_634L',
     'fortoday_1092L',
     'forweek_1077L',
     'forweek_528L',
     'forweek_601L',
     'foryear_618L',
     'foryear_818L',
     'foryear_850L',
     'fourthquarter_440L',
     'maritalst_385M',
     'maritalst_893M',
     'numberofqueries_373L',
     'pmtaverage_3A',
     'pmtaverage_4527227A',
     'pmtaverage_4955615A',
     'pmtcount_4527229L',
     'pmtcount_4955617L',
     'pmtcount_693L',
     'pmtscount_423L',
     'pmtssum_45A',
     'requesttype_4525192L',
     'responsedate_1012D',
     'responsedate_4527233D',
     'responsedate_4917613D',
     'riskassesment_302T',
     'riskassesment_940T',
     'secondquarter_766L',
     'thirdquarter_1082L'
]

static_0_1_cols = [
 'actualdpdtolerance_344P',
 'amtinstpaidbefduel24m_4187115A',
 'annuity_780A',
 'annuitynextmonth_57A',
 'applicationcnt_361L',
 'applications30d_658L',
 'applicationscnt_1086L',
 'applicationscnt_464L',
 'applicationscnt_629L',
 'applicationscnt_867L',
 'avgdbddpdlast24m_3658932P',
 'avgdbddpdlast3m_4187120P',
 'avgdbdtollast24m_4525197P',
 'avgdpdtolclosure24_3658938P',
 'avginstallast24m_3658937A',
 'avglnamtstart24m_4525187A',
 'avgmaxdpdlast9m_3716943P',
 'avgoutstandbalancel6m_4187114A',
 'avgpmtlast12m_4525200A',
 'bankacctype_710L',
 'cardtype_51L',
 'clientscnt12m_3712952L',
 'clientscnt3m_3712950L',
 'clientscnt6m_3712949L',
 'clientscnt_100L',
 'clientscnt_1022L',
 'clientscnt_1071L',
 'clientscnt_1130L',
 'clientscnt_136L',
 'clientscnt_157L',
 'clientscnt_257L',
 'clientscnt_304L',
 'clientscnt_360L',
 'clientscnt_493L',
 'clientscnt_533L',
 'clientscnt_887L',
 'clientscnt_946L',
 'cntincpaycont9m_3716944L',
 'cntpmts24_3658933L',
 'commnoinclast6m_3546845L',
 'credamount_770A',
 'credtype_322L',
 'currdebt_22A',
 'currdebtcredtyperange_828A',
 'datefirstoffer_1144D',
 'datelastinstal40dpd_247D',
 'datelastunpaid_3546854D',
 'daysoverduetolerancedd_3976961L',
 'deferredmnthsnum_166L',
 'disbursedcredamount_1113A',
 'disbursementtype_67L',
 'downpmt_116A',
 'dtlastpmtallstes_4499206D',
 'eir_270L',
 'equalitydataagreement_891L',
 'equalityempfrom_62L',
 'firstclxcampaign_1125D',
 'firstdatedue_489D',
 'homephncnt_628L',
 'inittransactionamount_650A',
 'inittransactioncode_186L',
 'interestrate_311L',
 'interestrategrace_34L',
 'isbidproduct_1095L',
 'isbidproductrequest_292L',
 'isdebitcard_729L',
 'lastactivateddate_801D',
 'lastapplicationdate_877D',
 'lastapprcommoditycat_1041M',
 'lastapprcommoditytypec_5251766M',
 'lastapprcredamount_781A',
 'lastapprdate_640D',
 'lastcancelreason_561M',
 'lastdelinqdate_224D',
 'lastdependentsnum_448L',
 'lastotherinc_902A',
 'lastotherlnsexpense_631A',
 'lastrejectcommoditycat_161M',
 'lastrejectcommodtypec_5251769M',
 'lastrejectcredamount_222A',
 'lastrejectdate_50D',
 'lastrejectreason_759M',
 'lastrejectreasonclient_4145040M',
 'lastrepayingdate_696D',
 'lastst_736L',
 'maininc_215A',
 'mastercontrelectronic_519L',
 'mastercontrexist_109L',
 'maxannuity_159A',
 'maxannuity_4075009A',
 'maxdbddpdlast1m_3658939P',
 'maxdbddpdtollast12m_3658940P',
 'maxdbddpdtollast6m_4187119P',
 'maxdebt4_972A',
 'maxdpdfrom6mto36m_3546853P',
 'maxdpdinstldate_3546855D',
 'maxdpdinstlnum_3546846P',
 'maxdpdlast12m_727P',
 'maxdpdlast24m_143P',
 'maxdpdlast3m_392P',
 'maxdpdlast6m_474P',
 'maxdpdlast9m_1059P',
 'maxdpdtolerance_374P',
 'maxinstallast24m_3658928A',
 'maxlnamtstart6m_4525199A',
 'maxoutstandbalancel12m_4187113A',
 'maxpmtlast3m_4525190A',
 'mindbddpdlast24m_3658935P',
 'mindbdtollast24m_4525191P',
 'mobilephncnt_593L',
 'monthsannuity_845L',
 'numactivecreds_622L',
 'numactivecredschannel_414L',
 'numactiverelcontr_750L',
 'numcontrs3months_479L',
 'numincomingpmts_3546848L',
 'numinstlallpaidearly3d_817L',
 'numinstls_657L',
 'numinstlsallpaid_934L',
 'numinstlswithdpd10_728L',
 'numinstlswithdpd5_4187116L',
 'numinstlswithoutdpd_562L',
 'numinstmatpaidtearly2d_4499204L',
 'numinstpaid_4499208L',
 'numinstpaidearly3d_3546850L',
 'numinstpaidearly3dest_4493216L',
 'numinstpaidearly5d_1087L',
 'numinstpaidearly5dest_4493211L',
 'numinstpaidearly5dobd_4499205L',
 'numinstpaidearly_338L',
 'numinstpaidearlyest_4493214L',
 'numinstpaidlastcontr_4325080L',
 'numinstpaidlate1d_3546852L',
 'numinstregularpaid_973L',
 'numinstregularpaidest_4493210L',
 'numinsttopaygr_769L',
 'numinsttopaygrest_4493213L',
 'numinstunpaidmax_3546851L',
 'numinstunpaidmaxest_4493212L',
 'numnotactivated_1143L',
 'numpmtchanneldd_318L',
 'numrejects9m_859L',
 'opencred_647L',
 'paytype1st_925L',
 'paytype_783L',
 'payvacationpostpone_4187118D',
 'pctinstlsallpaidearl3d_427L',
 'pctinstlsallpaidlat10d_839L',
 'pctinstlsallpaidlate1d_3546856L',
 'pctinstlsallpaidlate4d_3546849L',
 'pctinstlsallpaidlate6d_3546844L',
 'pmtnum_254L',
 'posfpd10lastmonth_333P',
 'posfpd30lastmonth_3976960P',
 'posfstqpd30lastmonth_3976962P',
 'previouscontdistrict_112M',
 'price_1097A',
 'sellerplacecnt_915L',
 'sellerplacescnt_216L',
 'sumoutstandtotal_3546847A',
 'sumoutstandtotalest_4493215A',
 'totaldebt_9A',
 'totalsettled_863A',
 'totinstallast1m_4525188A',
 'twobodfilling_608L',
 'typesuite_864L',
 'validfrom_1069D']

static_cb_0_cat = {
     'assignmentdate_238D',
     'assignmentdate_4527235D',
     'assignmentdate_4955616D',
     'birthdate_574D',
     'contractssum_5085716L',
     'dateofbirth_337D',
     'dateofbirth_342D',
     'description_5085714M',
     'education_1103M',
     'education_88M',
     'maritalst_385M',
     'maritalst_893M',
     'pmtcount_4527229L',
     'pmtcount_4955617L',
     'requesttype_4525192L',
     'responsedate_1012D',
     'responsedate_4527233D',
     'responsedate_4917613D',
     'riskassesment_302T'
}

static_0_1_cat = {
 'bankacctype_710L',
 'cardtype_51L',
 'clientscnt_136L',
 'credtype_322L',
 'datefirstoffer_1144D',
 'datelastinstal40dpd_247D',
 'datelastunpaid_3546854D',
 'disbursementtype_67L',
 'dtlastpmtallstes_4499206D',
 'equalitydataagreement_891L',
 'equalityempfrom_62L',
 'firstclxcampaign_1125D',
 'firstdatedue_489D',
 'inittransactioncode_186L',
 'isbidproduct_1095L',
 'isbidproductrequest_292L',
 'isdebitcard_729L',
 'lastactivateddate_801D',
 'lastapplicationdate_877D',
 'lastapprcommoditycat_1041M',
 'lastapprcommoditytypec_5251766M',
 'lastapprdate_640D',
 'lastcancelreason_561M',
 'lastdelinqdate_224D',
 'lastdependentsnum_448L',
 'lastrejectcommoditycat_161M',
 'lastrejectcommodtypec_5251769M',
 'lastrejectdate_50D',
 'lastrejectreason_759M',
 'lastrejectreasonclient_4145040M',
 'lastrepayingdate_696D',
 'lastst_736L',
 'maxdpdinstldate_3546855D',
 'opencred_647L',
 'paytype1st_925L',
 'paytype_783L',
 'payvacationpostpone_4187118D',
 'previouscontdistrict_112M',
 'twobodfilling_608L',
 'typesuite_864L',
 'validfrom_1069D'}

In [7]:
#############################################################################################
# TEST DATA SET // test_static_cb_0
#############################################################################################

### BASE TABLE
test_basetable = pl.read_csv(dataPath + "csv_files/test/test_base.csv")

### FEATURES
test_feature_set  = pl.concat(
    [
    pl.read_csv(dataPath + "csv_files/test/test_static_cb_0.csv").pipe(set_table_dtypes)
    ],
    how="vertical_relaxed"
)


#############################################################################################
# JOIN TABLES TOGETHER
#############################################################################################
test = test_basetable.join( 
    test_feature_set, how="left", on="case_id"
)

cols_pred = static_cb_0_cols
# for col in cols:
#      if col[-1].isupper() and col[:-1].islower():
#         cols_pred.append(col)
                
test_static = test[cols_pred].to_pandas()
# test_static = convert_strings(test_static)

numeric_cols = ['for3years_128L', 'for3years_504L', 'for3years_584L', 'formonth_118L',
                'formonth_206L', 'formonth_535L', 'forquarter_1017L', 'forquarter_462L',
                'forquarter_634L', 'fortoday_1092L', 'forweek_1077L', 'forweek_528L',
                'forweek_601L', 'foryear_618L', 'foryear_818L', 'foryear_850L', 'pmtcount_693L',
                'pmtscount_423L', 'riskassesment_940T']

for col in numeric_cols:
    test_static[col] = test_static[col].astype('float64')

categorical_cols = static_cb_0_cat

for col in categorical_cols:
    test_static[col] = test_static[col].astype('category')
    
pred_1 = model_static_cb_0.predict(test_static, num_iteration=model_static_cb_0.best_iteration)
pred_1

array([0.02129874, 0.02226254, 0.01866794, 0.0070892 , 0.02857628,
       0.01685833, 0.02519821, 0.01850613, 0.0541172 , 0.03872417])

In [20]:
#############################################################################################
# TEST DATA SET // test_static_0_1
#############################################################################################

### BASE TABLE
test_basetable = pl.read_csv(dataPath + "csv_files/test/test_base.csv")

### FEATURES
test_feature_set  = pl.concat(
    [
    pl.read_csv(dataPath + "csv_files/test/test_static_0_1.csv").pipe(set_table_dtypes)
    ],
    how="vertical_relaxed"
)


#############################################################################################
# JOIN TABLES TOGETHER
#############################################################################################
test = test_basetable.join( 
    test_feature_set, how="left", on="case_id"
)

cols_pred = static_0_1_cols
# for col in X_train.columns:
# #     if col[-1].isupper() and col[:-1].islower():
#         cols_pred.append(col)
                
test_static = test[cols_pred].to_pandas()
# test_static = convert_strings(test_static)

numeric_cols = ['deferredmnthsnum_166L', 'interestrategrace_34L']

for col in numeric_cols:
    test_static[col] = test_static[col].astype('float64')

categorical_cols = static_0_1_cat

for col in categorical_cols:
    test_static[col] = test_static[col].astype('category')
    

pred_2 = model_static_0_1.predict(test_static, num_iteration=model_static_0_1.best_iteration)
pred_2

array([0.0306813 , 0.05062306, 0.0306813 , 0.0306813 , 0.0306813 ,
       0.0306813 , 0.0306813 , 0.0306813 , 0.03570265, 0.00577467])

In [10]:
test_2 = pd.DataFrame({
    "case_id": pd.read_csv('/kaggle/input/home-credit-credit-risk-model-stability/csv_files/test/test_base.csv')["case_id"],
    "pred_1": pred_1,
    "pred_2": pred_2
})

test_2

,case_id,pred_1,pred_2
0,57543,0.021299,0.030681
1,57549,0.022263,0.050623
2,57551,0.018668,0.030681
3,57552,0.007089,0.030681
4,57569,0.028576,0.030681
5,57630,0.016858,0.030681
6,57631,0.025198,0.030681
7,57632,0.018506,0.030681
8,57633,0.054117,0.035703
9,57634,0.038724,0.005775


In [14]:
model_stage_2.predict_proba(test_2)[:, 1]
# score -> .352

[LightGBM] [Warning] min_data_in_leaf is set=9, min_child_samples=47 will be ignored. Current value: min_data_in_leaf=9
[LightGBM] [Warning] feature_fraction is set=0.5682663952386431, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5682663952386431
[LightGBM] [Warning] lambda_l2 is set=7.938707489172784, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.938707489172784
[LightGBM] [Warning] lambda_l1 is set=0.05640256237813216, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.05640256237813216
[LightGBM] [Warning] bagging_fraction is set=0.8735675970708932, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8735675970708932
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


array([0.01995965, 0.05376518, 0.00404875, 0.0034649 , 0.02447337,
       0.00659976, 0.02178482, 0.01301477, 0.06931771, 0.00274245])

In [42]:
# simple average of single predictions -> .39 score
(pred_1+pred_2)/2

array([0.02599002, 0.0364428 , 0.02467462, 0.01888525, 0.02962879,
       0.02376982, 0.02793976, 0.02459372, 0.04490993, 0.02224942])

In [40]:
# same thing with numpy
# # simple average -> .39 score
# np.mean([pred_1, pred_2], axis=0)

array([0.02599002, 0.0364428 , 0.02467462, 0.01888525, 0.02962879,
       0.02376982, 0.02793976, 0.02459372, 0.04490993, 0.02224942])

In [34]:
submission = pd.DataFrame({
    "case_id": pd.read_csv('/kaggle/input/home-credit-credit-risk-model-stability/csv_files/test/test_base.csv')["case_id"],
    "score": (pred_1+pred_2)/2, # score ~ .390
})

submission.to_csv('submission.csv')

In [35]:
submission

,case_id,score
0,57543,0.025990
1,57549,0.036443
2,57551,0.024675
3,57552,0.018885
4,57569,0.029629
5,57630,0.023770
6,57631,0.027940
7,57632,0.024594
8,57633,0.044910
9,57634,0.022249
